# Clustering baseado em índices acadêmicos
Ana's notebook

Essa análise consistem em três etapas: 
- Manipulação dos dados para cálculo do MC (média de conclusão) e da percentagem de aproveitamento de disciplinas cursadas de cada aluno
- Uso de algoritmo de clustering para gerar grupos de alunos 
- Uso do geocoder para colocar o endereço de cada aluno no mapa, cada grupo com uma dor diferente para observação de alguma relação

In [1]:
!pip install -U scikit-learn

from sklearn.cluster import AgglomerativeClustering, AffinityPropagation, KMeans

from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.palettes import Set1

# Import pandas as pd
import pandas as pd

#GEOCODER
import folium
import geocoder

Requirement already up-to-date: scikit-learn in /home/ana/anaconda3/lib/python3.6/site-packages


In [2]:
data = pd.read_csv('../data/imd-student-blind.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4842 entries, 0 to 4841
Data columns (total 11 columns):
Unnamed: 0            4842 non-null int64
a_ID                  4842 non-null int64
CEP                   4842 non-null int64
ano_ingresso          4842 non-null int64
periodo_ingresso      4842 non-null int64
status                4842 non-null object
ano_disciplina        4842 non-null int64
periodo_disciplina    4842 non-null int64
nota                  4842 non-null float64
disciplina_ID         4842 non-null int64
status.disciplina     4842 non-null object
dtypes: float64(1), int64(8), object(2)
memory usage: 416.2+ KB


In [4]:
data = data.sort_values(by='a_ID', ascending=True)
data.head()

,Unnamed: 0,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina
0,0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado
1400,1400,0,59015430,2014,1,CANCELADO,2014,2,1.0,6,Reprovado
1401,1401,0,59015430,2014,1,CANCELADO,2015,1,0.0,6,Reprovado
263,263,0,59015430,2014,1,CANCELADO,2014,1,7.1,1,Aprovado
686,686,0,59015430,2014,1,CANCELADO,2014,2,2.6,3,Reprovado


In [5]:
students = dict()
for index, row in data.iterrows():
    id = row['a_ID']
    if not id in students:
        students[id] = []
    students[id].append(row)

In [6]:
w = [60.0, 30.0, 180.0, 90.0, 30.0, 90.0, 60.0]
def weighted_average(grades, weights):
    l = len(grades)
    s = 0
    for g in range(0, l):
        s += grades[g]*weights[g]
    wa = s/sum(weights)
    return wa

def get_weight(index):
    return w[int(index)]

A função points gera uma lista composta por listas de dois elementos, sendo o primeiro elemento percentagem de aproveitamento e o segundo MC. 
Também retorna uma lista com os CEPs dos estudantes.

In [7]:
def points(students):
    p = []
    ceps = []
    for k, value in students.items():
        aprv = 0.0
        mc = 0.0
        grades = []
        weights = []
        for row in value:
            if row['status.disciplina'] == 'Aprovado':
                grades.append(row['nota'])
                weights.append(get_weight(row['disciplina_ID']))
                aprv+=1.0
        if len(grades) > 0:
            mc = weighted_average(grades, weights)
        if len(value) > 0:
            aprv = aprv/len(value)
        p.append([aprv, mc])
        ceps.append(value[0]['CEP'])
    return p, ceps

# Clustering

Agora utilzamos um método do pacote sklearn.cluster para gerar grupos de estudantes. 

Em mineração de dados, agrupamento k-means é um método de Clustering que objetiva particionar n observações dentre k grupos onde cada observação pertence ao grupo mais próximo da média. (fonte: Wikipédia)

Os dados são representados como pontos em um espaço euclidiano, onde o eixo x é a porcentagem de aproveitamento e no eixo y é o MC.

Fixamos o número de grupos em 3. 

In [8]:
n = 3 # number of clusterings
pts, ceps = points(students)
clusterings = KMeans(n_clusters=n, random_state=0).fit(pts)

In [9]:
plot = figure(x_axis_label ='aprov (perc)', y_axis_label ='mc')
labels = clusterings.labels_
x_axis = [None] * n
y_axis = [None] * n

for i in range(0, n):
    x_axis[i] = []
    y_axis[i] = []
    for p, l in zip(pts, labels):
        if l == i:
            x_axis[i].append(p[0])
            y_axis[i].append(p[1])
    plot.circle(x_axis[i], y_axis[i], color=Set1[n][i])
    
output_notebook()
show(plot)

Loading BokehJS ...

# Uso do geocoder

Na etapa anterior, utilizamos o kmeans para gerar os diferentes grupos de alunos, agora vamos utilizar os ceps para visualizar no mapa a distribuição desses grupos

In [ ]:
mapc = [0, 30]
zoom = 2
colors = ["red", "blue", "green", "orange", "gray", "yellow"]
map_students = folium.Map(location=mapc, zoom_start=zoom)

for cep, label in zip(ceps, labels):
    if cep > 0.001:
        g = geocoder.google(str(cep))
        lat = g.lat
        lng = g.lng
        if lat != None and lng != None: 
            folium.Marker([lat, lng], icon=folium.Icon(color=colors[label],icon='info-sign')).add_to(map_students)


In [ ]:
map_students